Afinación de parámetros con Keras Tuner
===

* 30:00 min | Última modificación: Mayo 10, 2021 | [YouTube]

Adaptado de:

* https://www.tensorflow.org/tutorials/keras/keras_tuner

In [2]:
import kerastuner as kt
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(kt.__version__)

2.4.1
1.0.2


In [3]:
#
#  Descarga de los datos
#
(
    (train_images, train_labels),
    (test_images, test_labels),
) = keras.datasets.fashion_mnist.load_data()

In [4]:
def model_builder(hp):

    model = keras.Sequential()

    inputs = keras.Input(shape=(28, 28))

    #
    # Capa de entrada
    #
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    #
    # Preprocesamiento
    #
    model.add(keras.layers.experimental.preprocessing.Rescaling(scale=1.0 / 255))

    #
    # Evalua distintas cantidades de neuronas en la capa
    # oculta
    #
    hp_units = hp.Int(
        "units",
        min_value=32,
        max_value=512,
        step=32,
    )
    model.add(
        keras.layers.Dense(
            units=hp_units,
            activation="relu",
        )
    )

    #
    # La cantidad de neuronas en la capa de salida
    # permanece fija
    #
    model.add(keras.layers.Dense(10))

    #
    # Escoge el mejor valor para la tasa de aprendizaje
    #
    hp_learning_rate = hp.Choice(
        "learning_rate",
        values=[0.01, 0.001, 0.0001],
    )

    model.compile(
        #
        # Optimizador con distintos valores 
        # para la tasa de aprendizaje
        #
        optimizer=keras.optimizers.Adam(
            learning_rate=hp_learning_rate,
        ),
        #
        # Función de pérdida
        #
        loss=keras.losses.SparseCategoricalCrossentropy(
            from_logits=True,
        ),
        #
        # Métrica a monitoreas
        #
        metrics=["accuracy"],
    )

    return model

In [6]:
#
# Existen cuantro funciones:
#
# - Hyperband
# - RandomSearch
# - BayesianOptimization
# - Sklearn
#
tuner = kt.Hyperband(
    #
    # Construye el modelo
    #
    model_builder,
    #
    # Función usada para seleccionar los
    # mejores parámetros
    #
    objective="val_accuracy",
    #
    # Número máximo de iteraciones
    #
    max_epochs=10,
    #
    #
    #
    factor=3,
    #
    # directorio de trabajo
    #
    directory="my_dir",
    #
    # Nombre del proyecto
    #
    project_name="intro_to_kt",
)

#
#
#
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
)

tuner.search(
    train_images,
    train_labels,
    epochs=50,
    validation_split=0.2,
    callbacks=[stop_early],
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


#
# Cantidad optima de neuronas en la capa oculta
#
best_hps.get("units")

Trial 30 Complete [00h 00m 29s]
val_accuracy: 0.8568333387374878

Best val_accuracy So Far: 0.8896666765213013
Total elapsed time: 00h 06m 52s
INFO:tensorflow:Oracle triggered exit


448

In [7]:
#
# Tasa de aprendizaje optima
#
best_hps.get('learning_rate')

0.001

In [11]:
#
# Construcción del modelo con los hiperparametros óptimos
# y entrenamiento para 50 epochs
#
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    train_images,
    train_labels,
    epochs=50,
    validation_split=0.2,
)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
best_epoch

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6104 - accuracy: 0.7885 - val_loss: 0.4219 - val_accuracy: 0.8404
Epoch 2/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3790 - accuracy: 0.8627 - val_loss: 0.3559 - val_accuracy: 0.8748
Epoch 3/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3311 - accuracy: 0.8781 - val_loss: 0.3441 - val_accuracy: 0.8766
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3102 - accuracy: 0.8851 - val_loss: 0.3366 - val_accuracy: 0.8786
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2888 - accuracy: 0.8925 - val_loss: 0.3403 - val_accuracy: 0.8762
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2671 - accuracy: 0.9002 - val_loss: 0.3177 - val_accuracy: 0.8841
Epoch 7/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2561 - accuracy: 0.9035 - val_loss: 0.3175 - val_accuracy:

42

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(
    train_images,
    train_labels,
    epochs=best_epoch,
    validation_split=0.2,
)

Epoch 1/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6137 - accuracy: 0.7862 - val_loss: 0.4041 - val_accuracy: 0.8563
Epoch 2/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3783 - accuracy: 0.8626 - val_loss: 0.3585 - val_accuracy: 0.8727
Epoch 3/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3346 - accuracy: 0.8756 - val_loss: 0.3700 - val_accuracy: 0.8694
Epoch 4/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3041 - accuracy: 0.8888 - val_loss: 0.3515 - val_accuracy: 0.8750
Epoch 5/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2890 - accuracy: 0.8921 - val_loss: 0.3306 - val_accuracy: 0.8815
Epoch 6/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2743 - accuracy: 0.8986 - val_loss: 0.3414 - val_accuracy: 0.8814
Epoch 7/42
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2602 - accuracy: 0.9025 - val_loss: 0.3205 - val_accuracy:

In [15]:
hypermodel.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.5104 - accuracy: 0.8917


[0.5104374289512634, 0.891700029373169]